In [5]:
credkey = "5l1CcEezvl3tKMg9IrCxxpp7aqyzu958LKJRbuykcpbLw936K1eoJQQJ99ALACYeBjFXJ3w3AAAAACOGkpIM"
endpoint = "https://rag-lab-ais413302541331.cognitiveservices.azure.com/"

In [6]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest

In [7]:
def _in_span(word, spans):
    for span in spans:
        if word.span.offset >= span.offset and (word.span.offset + word.span.length) <= (span.offset + span.length):
            return True
        
def get_words(page, line):
    result = []

    for word in page.words:
        if _in_span(word, line.spans):
            result.append(word)

    return result

In [12]:
doc_url = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"

client = DocumentIntelligenceClient(
    endpoint=endpoint, 
    credential=AzureKeyCredential(credkey)
)

poller = client.begin_analyze_document(
    model_id="prebuilt-layout", 
    body=AnalyzeDocumentRequest(url_source=doc_url),
    output_content_format="markdown"
)

result = poller.result()

In [13]:
if result.styles and any([style.is_handwritten for style in result.styles]):
    print("Document contains handwritten content.")
else:
    print("Document does not contain handwritten content.")

Document does not contain handwritten content.


In [14]:
for page in result.pages:
    print(f"Analyzing layout from page #{page.page_number}")
    print(f"Page has width: {page.width} and height: {page.height}, measured with unit: {page.unit}")

    if page.lines:
        for line_idx, line in enumerate(page.lines):
            print(f"    Line {line_idx} contains '{line.content}' within '{line.polygon}")

            words = get_words(page, line)

            for word in words:
                print(f"    Word '{word.content}' has confidence {word.confidence}.")
    
    if page.selection_marks:
        for selection_mark in page.selection_marks:
            print(f"    Selection mark is '{selection_mark.state}' at '{selection_mark.polygon}' with confidence {selection_mark.confidence}.")

Analyzing layout from page #1
Page has width: 8.5 and height: 11.0, measured with unit: LengthUnit.INCH
    Line 0 contains 'UNITED STATES' within '[3.4695, 0.6555, 5.0216, 0.6576, 5.0214, 0.847, 3.4693, 0.845]
    Word 'UNITED' has confidence 0.997.
    Word 'STATES' has confidence 0.998.
    Line 1 contains 'SECURITIES AND EXCHANGE COMMISSION' within '[2.1754, 0.8727, 6.3155, 0.8723, 6.3155, 1.0737, 2.1754, 1.0742]
    Word 'SECURITIES' has confidence 0.993.
    Word 'AND' has confidence 0.998.
    Word 'EXCHANGE' has confidence 0.992.
    Word 'COMMISSION' has confidence 0.992.
    Line 2 contains 'Washington, D.C. 20549' within '[3.4515, 1.0922, 5.0382, 1.0888, 5.0386, 1.2532, 3.4518, 1.2565]
    Word 'Washington,' has confidence 0.995.
    Word 'D.C.' has confidence 0.993.
    Word '20549' has confidence 0.995.
    Line 3 contains 'FORM 10-Q' within '[3.7116, 1.3956, 4.7861, 1.3942, 4.7863, 1.5875, 3.7118, 1.5888]
    Word 'FORM' has confidence 0.992.
    Word '10-Q' has confidenc

In [11]:
if result.tables:
    for table_idx, table in enumerate(result.tables):
        print(f"Table {table_idx} has {table.row_count} rows and {table.column_count} columns.")

        if table.bounding_regions:
            for region in table.bounding_regions:
                print(f"    Table is located at '{region.polygon}' on page {region.page_number}.")
        for cell in table.cells:
            print(f"    Cell at row {cell.row_index}, column {cell.column_index} contains '{cell.content}'.")
            if cell.bounding_regions:
                for region in cell.bounding_regions:
                    print(f"        Cell is located at '{region.polygon}' on page {region.page_number}.")

Table 0 has 5 rows and 3 columns.
    Table is located at '[0.5912, 4.9025, 6.9694, 4.9034, 6.9718, 5.7826, 0.5926, 5.782]' on page 1.
    Cell at row 0, column 0 contains 'Title of each class'.
        Cell is located at '[0.5702, 4.8825, 3.8454, 4.8825, 3.8389, 5.1255, 0.5636, 5.1255]' on page 1.
    Cell at row 0, column 1 contains 'Trading Symbol'.
        Cell is located at '[3.8454, 4.8825, 5.3713, 4.8825, 5.3713, 5.1255, 3.8389, 5.1255]' on page 1.
    Cell at row 0, column 2 contains 'Name of exchange on which registered'.
        Cell is located at '[5.3713, 4.8825, 6.93, 4.8825, 6.93, 5.1123, 5.3713, 5.1255]' on page 1.
    Cell at row 1, column 0 contains 'Common stock, $0.00000625 par value per share'.
        Cell is located at '[0.5636, 5.1255, 3.8389, 5.1255, 3.8389, 5.2962, 0.5636, 5.2962]' on page 1.
    Cell at row 1, column 1 contains 'MSFT'.
        Cell is located at '[3.8389, 5.1255, 5.3713, 5.1255, 5.3713, 5.2962, 3.8389, 5.2962]' on page 1.
    Cell at row 1, co

In [15]:
result.content

'# UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549\n\n\n## FORM 10-Q\n\n☐\n☒\n☒\nQUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF\n1934\nFor the Quarterly Period Ended March 31, 2020\nOR\n☐\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF\n1934\nFor the Transition Period From\nto\n\nCommission File Number 001-37845\n\n\n## MICROSOFT CORPORATION\n\nWASHINGTON\n(STATE OF INCORPORATION)\nONE MICROSOFT WAY, REDMOND, WASHINGTON 98052-6399\n(425) 882-8080\nwww.microsoft.com/investor\n\n91-1144442\n(I.R.S. ID)\n\nSecurities registered pursuant to Section 12(b) of the Act:\n\n\n<table>\n<tr>\n<th>Title of each class</th>\n<th>Trading Symbol</th>\n<th>Name of exchange on which registered</th>\n</tr>\n<tr>\n<td>Common stock, $0.00000625 par value per share</td>\n<td>MSFT</td>\n<td>NASDAQ</td>\n</tr>\n<tr>\n<td>2.125% Notes due 2021</td>\n<td>MSFT</td>\n<td>NASDAQ</td>\n</tr>\n<tr>\n<td>3.125% Notes due 